In [9]:
import pandas as pd
essays = pd.read_csv('trained.csv')
essays.set_index('essay_id',inplace=True, drop=True)
X = essays.drop(['domain1_score','essay','essay_set','normal_score','n_top500'], axis=1)
y = essays['normal_score']
X.head()

,misspelt,correct,length,lexical_diversity,n_sentences,nouns,verbs,adverbs,adjectives,wordnetscore
essay_id,,,,,,,,,,
14834,6,116,123,1.37,9,28,14,8,10,3
14835,11,169,180,1.55,10,41,23,12,14,6
14836,3,163,169,1.62,9,39,23,10,9,24
14837,18,175,199,1.69,11,35,26,7,4,4
14838,4,158,162,1.74,11,35,26,8,15,8


In [10]:
from sklearn.model_selection import train_test_split

# Split X and y into X_
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)


In [11]:
from sklearn.linear_model import LinearRegression

regression_model = LinearRegression()
regression_model.fit(X_train, y_train)

LinearRegression()

In [15]:
regression_model.intercept_

0.9802623621004534

In [16]:
regression_model.coef_

array([ 0.04436805,  0.04869135, -0.05002495, -0.16109472,  0.0214928 ,
        0.02224063,  0.00851278,  0.02867632,  0.03380014,  0.10293278])

In [29]:
from sklearn.metrics import r2_score,accuracy_score
print(regression_model.score(X_test, y_test))
y_pred = regression_model.predict(X_test)

0.5476631368037861


In [18]:
from sklearn.metrics import mean_squared_error

y_predict = regression_model.predict(X_test)

regression_model_mse = mean_squared_error(y_predict, y_test)

regression_model_mse
import math

math.sqrt(regression_model_mse)

0.6615578668610803

In [28]:
from sklearn.preprocessing import PolynomialFeatures
for val in range(1,10):
    poly_reg = PolynomialFeatures(degree=val)
    X_poly = poly_reg.fit_transform(X)
    pol_reg = LinearRegression()
    pol_reg.fit(X_poly, y)
    y_predict = pol_reg.predict(X_poly)
    print("rmse = {}",math.sqrt(mean_squared_error(y_predict, y)))
    print("r2_score = {}",r2_score(y,y_predict))

rmse = {} 0.6310288655954812
r2_score = {} 0.5771055339690739
rmse = {} 0.5902978010398556
r2_score = {} 0.6299368161507126
rmse = {} 0.5428909344805118
r2_score = {} 0.686989627505179
rmse = {} 0.3777662785806713
r2_score = {} 0.848441629956787
rmse = {} 0.0007012951920560749
r2_score = {} 0.9999994776816628
rmse = {} 0.003949987770746355
r2_score = {} 0.9999834299029429
rmse = {} 0.01067836787495668
r2_score = {} 0.9998789002331426
rmse = {} 0.03620256990743142
r2_score = {} 0.9986080861640798
rmse = {} 0.07564082459258668
r2_score = {} 0.9939236041365267


In [ ]:
for val in range(1,10):
    poly_reg = PolynomialFeatures(degree=val)
    X_poly = poly_reg.fit_transform(X_train)
    pol_reg = LinearRegression()
    pol_reg.fit(X_poly, y_train)
    y_predict = pol_reg.predict(poly_reg.transform(X_test))
    print("rmse = {}",math.sqrt(mean_squared_error(y_predict, y_test)))
    print("r2_score = {}",r2_score(y_test,y_predict))

rmse = {} 0.6615578668610803
r2_score = {} 0.5476631368037861
rmse = {} 0.652343096762048
r2_score = {} 0.560176482458684
rmse = {} 0.7656599505085565
r2_score = {} 0.3941038852743344
rmse = {} 5.973572184830538
r2_score = {} -35.88030787221223
rmse = {} 90.61313557393319
r2_score = {} -8485.10934856172
rmse = {} 1450.9117135240137
r2_score = {} -2175745.4290091866
rmse = {} 28155.2673024691
r2_score = {} -819305017.996069
